In [29]:
import pandas as pd
from transformers import BertTokenizer

In [30]:
llama_data = {}
for i in range(5):
    llama_data[i] = pd.read_csv(f"data/{i}_shot_data.tsv", sep="\t", decimal=",", index_col=0)
    llama_data[i]["num_unique"] = llama_data[i].apply(lambda row: int(row["num_generated"] * row["unique_generated"]), axis=1)

In [31]:
mbart_data = pd.read_csv("data/data_mbart_complete2.tsv", sep="\t", index_col=0)
mbart_vanilla_data = pd.read_csv("data/data_mbart_vanilla_complete2.tsv", sep="\t", index_col=0)

In [32]:
bsbbert_data = pd.read_csv("data/data_bsbbert_complete.tsv", sep="\t", index_col=0)

In [33]:
bsbbert_data

,construction,pseudoword,example,generated,score,definition
0,5,Und5,['Und schon gar nicht mit der Mehrwertsteuer ....,"[['[CLS]', 'Und5', 'schon', 'gar', 'nicht', 'b...","[14.645186424255371, 15.811990737915039, 9.996...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr..."
1,5,erst5,['Trainer Lucien Favre hatte schon seine beide...,"[['[CLS]', 'Trainer', 'Luc', '##ien', 'Fa', '#...","[15.751483917236328, 11.782296180725098, 9.956...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr..."
2,5,gar5,['Es hat Afghanistan nicht stabilisiert und sc...,"[['[CLS]', 'Das', 'hat', 'Afghanistan', 'nicht...","[11.360899925231934, 14.937478065490723, 15.68...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr..."
3,5,nicht5,['Es hat Afghanistan nicht stabilisiert und sc...,"[""'[MASK]' is not in list""]",[-1.0],"Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr..."
4,5,recht5,['Trainer Lucien Favre hatte schon seine beide...,"[['[CLS]', 'Trainer', 'Luc', '##ien', 'Fa', '#...","[15.844961166381836, 11.985773086547852, 10.04...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr..."
...,...,...,...,...,...,...
576,1884,Gold1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","[['[CLS]', 'Reden', 'ist', 'Silber', ',', 'red...","[11.769918441772461, 10.454679489135742, 15.89...","Die ""Intensivierung_Komparativ:Xist_SilberYist..."
577,1884,Silber1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","[['[CLS]', 'Reden', 'ist', 'Silber1884', ',', ...","[12.572469711303711, 9.817983627319336, 14.939...","Die ""Intensivierung_Komparativ:Xist_SilberYist..."
578,1884,ist1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","[['[CLS]', 'Reden', 'ist1884', 'Silber', ',', ...","[14.046998977661133, 12.147357940673828, 13.26...","Die ""Intensivierung_Komparativ:Xist_SilberYist..."
579,1986,kaum1986,['Die Vorhut vor 20.000 Jahren war für das Rie...,"[['[CLS]', 'Die', 'Vor', '##hut', 'vor', '20',...","[14.439410209655762, 13.822213172912598, 18.17...","Die Konstruktion ""Relativierung:kaumADJ"" gehör..."


In [34]:
def evaluate_generated(row):
    try:
        new = eval(row["generated"])
    except:
        new = []
    return new

In [35]:
mbart_data["evaluated_generated"] = mbart_data.apply(evaluate_generated, axis=1)
mbart_data

,construction,pseudoword,orig_example,generated,scores,evaluated_generated
0,5,Und5,"['""""Nicht herauskaufen"""", """"nicht erpressen la...","['... Mehrwertsteuer.......', 'Auf das schnell...","[0.7702094316482544, 0.9311211109161377, 0.736...","[... Mehrwertsteuer......., Auf das schnelle V..."
1,5,erst5,['Trainer Lucien Favre hatte schon seine beide...,"['...........', 'Tatsächlich sind wir uns niem...","[0.6238253712654114, 0.7992855310440063, 0.701...","[..........., Tatsächlich sind wir uns niemals..."
2,5,gar5,['Es hat Afghanistan nicht stabilisiert und sc...,['Es hat Afghanistan nicht stabilisiert und sc...,"[0.7928561568260193, 0.8251732587814331, 0.658...",[Es hat Afghanistan nicht stabilisiert und sch...
3,5,nicht5,['Es hat Afghanistan nicht stabilisiert und sc...,NaN,[-1.0],[]
4,5,recht5,['Trainer Lucien Favre hatte schon seine beide...,"['............', 'Tatsächlich sind wir uns nie...","[0.6184999942779541, 0.8167852759361267, 0.695...","[............, Tatsächlich sind wir uns niemal..."
...,...,...,...,...,...,...
501,1884,Gold1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","['Schweigen ist Silber, reden ist Gold.....', ...","[0.8336131572723389, 0.8371401429176331, 0.801...","[Schweigen ist Silber, reden ist Gold....., Sc..."
502,1884,Silber1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","['Schweigen ist... Gold...', 'Schweigen ist Go...","[0.7981865406036377, 0.7699515223503113, 0.754...","[Schweigen ist... Gold..., Schweigen ist Gold!..."
503,1884,ist1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","['Schweigen... Gold....', 'Schweigen Gold!', '...","[0.8048365116119385, 0.7254974246025085, 0.690...","[Schweigen... Gold...., Schweigen Gold!, Angeb..."
504,1986,kaum1986,['Die Vorhut vor 20.000 Jahren war für das Rie...,['Die Vorhut vor 20.000 Jahren war für das Rie...,"[0.8100022673606873, 0.847483217716217, 0.6762...",[Die Vorhut vor 20.000 Jahren war für das Ries...


In [36]:
mbart_vanilla_data["evaluated_generated"] = mbart_vanilla_data.apply(evaluate_generated, axis=1)
mbart_vanilla_data

,construction,pseudoword,orig_example,generated,scores,evaluated_generated
0,5,Und5,"['""""Nicht herauskaufen"""", """"nicht erpressen la...","['""""Nicht herauskaufen"""", """"nicht erpressen la...","[0.623329758644104, 0.7097308039665222, 0.5912...","[""""Nicht herauskaufen"""", """"nicht erpressen las..."
1,5,erst5,['Trainer Lucien Favre hatte schon seine beide...,['Trainer Lucien Favre hat seine beiden Stürme...,"[0.4603891670703888, 0.6319332718849182, 0.565...",[Trainer Lucien Favre hat seine beiden Stürmer...
2,5,gar5,['Es hat Afghanistan nicht stabilisiert und sc...,['Es hat Afghanistan nicht stabilisiert und sc...,"[0.9639642238616943, 0.4984430968761444, 0.944...",[Es hat Afghanistan nicht stabilisiert und sch...
3,5,nicht5,['Es hat Afghanistan nicht stabilisiert und sc...,"['Es hat Afghanistan nicht befriedet.', ', abe...","[0.7552201747894287, 0.4845995604991913, 0.874...","[Es hat Afghanistan nicht befriedet., , aber n..."
4,5,recht5,['Trainer Lucien Favre hatte schon seine beide...,['Trainer Lucien Favre hat seine beiden Stürme...,"[0.4634501338005066, 0.5811089873313904, 0.563...",[Trainer Lucien Favre hat seine beiden Stürmer...
...,...,...,...,...,...,...
501,1884,Gold1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","['Schweigen ist Silber, reden ist Gold', 'Schw...","[0.8799072504043579, 0.8578562140464783, 0.912...","[Schweigen ist Silber, reden ist Gold, Schweig..."
502,1884,Silber1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","['Schweigen ist Silber, Schweigen ist Gold.', ...","[0.8501437306404114, 0.8222342729568481, 0.793...","[Schweigen ist Silber, Schweigen ist Gold., Sc..."
503,1884,ist1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","['Schweigen ist wie Gold.', 'Schweigen ist wie...","[0.7324506044387817, 0.7599738836288452, 0.599...","[Schweigen ist wie Gold., Schweigen ist wie Go..."
504,1986,kaum1986,['Die Vorhut vor 20.000 Jahren war für das Rie...,['Die Vorhut vor 20.000 Jahren war für das Rie...,"[0.9354256987571716, 0.5056698322296143, 0.492...",[Die Vorhut vor 20.000 Jahren war für das Ries...


In [37]:
for llama in llama_data.values():
    llama["evaluated_generated"] = llama.apply(evaluate_generated, axis=1)
llama_data[0]

,definition,examples,generated,evaluated_generated,num_generated,unique_generated,definition_overlap,examples_overlap,overlap,num_unique
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e...",[Die meisten von ihnen haben zuvor noch nie ei...,2,1.0,1.000000,0.0,1.000000,2
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die beiden Pferde sind gleicher Größe."", ""Di...","[Die beiden Pferde sind gleicher Größe., Die b...",2,1.0,0.166667,0.0,0.166667,2
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Michael is the tallest"", ""John has the short...","[Michael is the tallest, John has the shortest...",3,1.0,0.000000,0.0,0.000000,3
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""The athlete is the fastest runner on the tea...",[The athlete is the fastest runner on the team.],1,1.0,0.000000,0.0,0.000000,1
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt wie eine Lokomotive"", ""Das Ha...","[Das Auto fährt wie eine Lokomotive, Das Haus ...",3,1.0,0.000000,0.0,0.000000,3
...,...,...,...,...,...,...,...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der höher ist als der Frau"", ""Ein ...","[Ein Mann, der höher ist als der Frau, Ein Man...",3,1.0,0.142857,0.0,0.142857,3
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""Xoder"", ""Y""]","[Xoder, Y]",2,1.0,0.000000,0.0,0.000000,2
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo das Licht ist, ist auch die Dunkelheit"", ...","[Wo das Licht ist, ist auch die Dunkelheit, Wo...",4,1.0,0.222222,0.0,0.222222,4
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""Die 'äquativ' ist gleich 22"", ""Das Bild ist ...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",2,1.0,0.000000,0.0,0.000000,2


In [38]:
bsbbert_data["evaluated_generated"] = bsbbert_data.apply(evaluate_generated, axis=1)
bsbbert_data

,construction,pseudoword,example,generated,score,definition,evaluated_generated
0,5,Und5,['Und schon gar nicht mit der Mehrwertsteuer ....,"[['[CLS]', 'Und5', 'schon', 'gar', 'nicht', 'b...","[14.645186424255371, 15.811990737915039, 9.996...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr...","[[[CLS], Und5, schon, gar, nicht, bei, der, Me..."
1,5,erst5,['Trainer Lucien Favre hatte schon seine beide...,"[['[CLS]', 'Trainer', 'Luc', '##ien', 'Fa', '#...","[15.751483917236328, 11.782296180725098, 9.956...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr...","[[[CLS], Trainer, Luc, ##ien, Fa, ##vre, hatte..."
2,5,gar5,['Es hat Afghanistan nicht stabilisiert und sc...,"[['[CLS]', 'Das', 'hat', 'Afghanistan', 'nicht...","[11.360899925231934, 14.937478065490723, 15.68...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr...","[[[CLS], Das, hat, Afghanistan, nicht, stabil,..."
3,5,nicht5,['Es hat Afghanistan nicht stabilisiert und sc...,"[""'[MASK]' is not in list""]",[-1.0],"Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr...",['[MASK]' is not in list]
4,5,recht5,['Trainer Lucien Favre hatte schon seine beide...,"[['[CLS]', 'Trainer', 'Luc', '##ien', 'Fa', '#...","[15.844961166381836, 11.985773086547852, 10.04...","Die ""Negation:NEG_Xund_schon_gar_nichtY-Konstr...","[[[CLS], Trainer, Luc, ##ien, Fa, ##vre, hatte..."
...,...,...,...,...,...,...,...
576,1884,Gold1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","[['[CLS]', 'Reden', 'ist', 'Silber', ',', 'red...","[11.769918441772461, 10.454679489135742, 15.89...","Die ""Intensivierung_Komparativ:Xist_SilberYist...","[[[CLS], Reden, ist, Silber, ,, reden, ist, Go..."
577,1884,Silber1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","[['[CLS]', 'Reden', 'ist', 'Silber1884', ',', ...","[12.572469711303711, 9.817983627319336, 14.939...","Die ""Intensivierung_Komparativ:Xist_SilberYist...","[[[CLS], Reden, ist, Silber1884, ,, reden, ist..."
578,1884,ist1884,"['Schweigen ist Silber , reden ist Gold .', 'S...","[['[CLS]', 'Reden', 'ist1884', 'Silber', ',', ...","[14.046998977661133, 12.147357940673828, 13.26...","Die ""Intensivierung_Komparativ:Xist_SilberYist...","[[[CLS], Reden, ist1884, Silber, ,, reden, ist..."
579,1986,kaum1986,['Die Vorhut vor 20.000 Jahren war für das Rie...,"[['[CLS]', 'Die', 'Vor', '##hut', 'vor', '20',...","[14.439410209655762, 13.822213172912598, 18.17...","Die Konstruktion ""Relativierung:kaumADJ"" gehör...","[[[CLS], Die, Vor, ##hut, vor, 20, ., 000, Jah..."


In [39]:
llama_subset = pd.concat((llama_data[0][["definition", "evaluated_generated"]], llama_data[4]["examples"]), axis=1)
llama_subset

,definition,evaluated_generated,examples
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[Die meisten von ihnen haben zuvor noch nie ei...,"['Oder man geht ins Studio , wo man Songs aufn..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...","[Die beiden Pferde sind gleicher Größe., Die b...",['Sicherheitsgefühl und objektive Sicherheit s...
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...","[Michael is the tallest, John has the shortest...","['Gut , besser , "" "" Les Bleus "" "" :', '23. Ju..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[The athlete is the fastest runner on the team.],['ZEIT : Die Aufnahmen zu Bowies letzten Platt...
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...","[Das Auto fährt wie eine Lokomotive, Das Haus ...","['Ein Spiel , von dem sich viele unserer Leser..."
...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...","[Ein Mann, der höher ist als der Frau, Ein Man...","['Nämlich cooler als cool , bitteschön .', 'We..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...","[Xoder, Y]","['Weinen wie die Holländer , oder : Wenn Angel..."
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...","[Wo das Licht ist, ist auch die Dunkelheit, Wo...","['Wo Energie ist , ist Wohlstand .', 'Wo viel ..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",['Baum : Ich mache das natürlich nicht mehr so...


In [40]:
llama_subset_3 = pd.concat((llama_data[3][["definition", "evaluated_generated"]], llama_data[4]["examples"]), axis=1)
llama_subset_3

,definition,evaluated_generated,examples
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[Die meisten von ihnen haben zuvor noch nie ei...,"['Oder man geht ins Studio , wo man Songs aufn..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[Die Schülerzahlen im Jahrgang A und B sind gl...,['Sicherheitsgefühl und objektive Sicherheit s...
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...","[Besser, Die besten, Groß, Größer, Nie komplex...","['Gut , besser , "" "" Les Bleus "" "" :', '23. Ju..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[Er hat die Sache mit höchster Wichtigkeit erl...,['ZEIT : Die Aufnahmen zu Bowies letzten Platt...
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...","[Er hat sich hart trainiert, um ein marathonfä...","['Ein Spiel , von dem sich viele unserer Leser..."
...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...","[Habt ihr eine Chance gehabt, in einem warmen ...","['Nämlich cooler als cool , bitteschön .', 'We..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...","[Nietzsche - oder der Wille zur Macht, Theorie...","['Weinen wie die Holländer , oder : Wenn Angel..."
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"['Wo Energie ist , ist Wohlstand .', 'Wo viel ..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...","[Ich habe zuvor nur so geschaltet, Ich lese ni...",['Baum : Ich mache das natürlich nicht mehr so...


In [41]:
mbart_merged = pd.merge(mbart_vanilla_data[["construction", "evaluated_generated"]], mbart_data[["construction", "evaluated_generated"]], on="construction", how="inner", suffixes=["_mbart_vanilla", "_mbart_pseudo"])
mbart_merged

,construction,evaluated_generated_mbart_vanilla,evaluated_generated_mbart_pseudo
0,5,"[""""Nicht herauskaufen"""", """"nicht erpressen las...","[... Mehrwertsteuer......., Auf das schnelle V..."
1,5,"[""""Nicht herauskaufen"""", """"nicht erpressen las...","[..........., Tatsächlich sind wir uns niemals..."
2,5,"[""""Nicht herauskaufen"""", """"nicht erpressen las...",[Es hat Afghanistan nicht stabilisiert und sch...
3,5,"[""""Nicht herauskaufen"""", """"nicht erpressen las...",[]
4,5,"[""""Nicht herauskaufen"""", """"nicht erpressen las...","[............, Tatsächlich sind wir uns niemal..."
...,...,...,...
2827,1884,"[Schweigen ist wie Gold., Schweigen ist wie Go...","[Schweigen ist Silber, reden ist Gold....., Sc..."
2828,1884,"[Schweigen ist wie Gold., Schweigen ist wie Go...","[Schweigen ist... Gold..., Schweigen ist Gold!..."
2829,1884,"[Schweigen ist wie Gold., Schweigen ist wie Go...","[Schweigen... Gold...., Schweigen Gold!, Angeb..."
2830,1986,[Die Vorhut vor 20.000 Jahren war für das Ries...,[Die Vorhut vor 20.000 Jahren war für das Ries...


In [42]:
final_merged = pd.merge(llama_subset, mbart_merged, left_index=True, right_on="construction")
final_merged = pd.merge(final_merged, bsbbert_data[["construction", "evaluated_generated"]], on="construction", suffixes=["_llama", "_bert"])
final_merged = pd.merge(final_merged, llama_subset_3["evaluated_generated"], left_on="construction", right_index=True)
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
final_merged["evaluated_generated_bert"] = final_merged.apply(lambda row: [tokenizer.convert_tokens_to_string([" ".join([str(s) for s in n])]) for n in row["evaluated_generated_bert"]], axis=1)
final_merged.rename({"evaluated_generated": "evaluated_generated_llama3"}, axis=1, inplace=True)

final_merged

,definition,evaluated_generated_llama,examples,construction,evaluated_generated_mbart_vanilla,evaluated_generated_mbart_pseudo,evaluated_generated_bert,evaluated_generated_llama3
0,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[Die meisten von ihnen haben zuvor noch nie ei...,"['Oder man geht ins Studio , wo man Songs aufn...",10,"[Die Chance dazu haben sie, und sie haben sie ...","[Die Chance dazu haben sie, und sie haben sie ...","[[CLS] Geschweige10 denn , dass sie die Argume...",[Die meisten von ihnen haben zuvor noch nie ei...
1,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[Die meisten von ihnen haben zuvor noch nie ei...,"['Oder man geht ins Studio , wo man Songs aufn...",10,"[Die Chance dazu haben sie, und sie haben sie ...","[Die Chance dazu haben sie, und sie haben sie ...",[[CLS] Und dann ist da noch das generelle Prob...,[Die meisten von ihnen haben zuvor noch nie ei...
2,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[Die meisten von ihnen haben zuvor noch nie ei...,"['Oder man geht ins Studio , wo man Songs aufn...",10,"[Die Chance dazu haben sie, und sie haben sie ...","[Die Chance dazu haben sie, und sie haben sie ...",[[CLS] Und dann ist da noch das generelle Prob...,[Die meisten von ihnen haben zuvor noch nie ei...
3,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[Die meisten von ihnen haben zuvor noch nie ei...,"['Oder man geht ins Studio , wo man Songs aufn...",10,"[Die Chance dazu haben sie, und sie haben sie ...","[..........., »Ohne Europa sind viele Fragen n...","[[CLS] Geschweige10 denn , dass sie die Argume...",[Die meisten von ihnen haben zuvor noch nie ei...
4,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[Die meisten von ihnen haben zuvor noch nie ei...,"['Oder man geht ins Studio , wo man Songs aufn...",10,"[Die Chance dazu haben sie, und sie haben sie ...","[..........., »Ohne Europa sind viele Fragen n...",[[CLS] Und dann ist da noch das generelle Prob...,[Die meisten von ihnen haben zuvor noch nie ei...
...,...,...,...,...,...,...,...,...
28307,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",['Baum : Ich mache das natürlich nicht mehr so...,98,[Als Vorstandsvorsitzender verdient er grob 10...,[Als Vorstandsvorsitzender verdient er grob 10...,[[CLS] Nun bleibt die Frage : Wird der Bitcoin...,"[Ich habe zuvor nur so geschaltet, Ich lese ni..."
28308,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",['Baum : Ich mache das natürlich nicht mehr so...,98,[Als Vorstandsvorsitzender verdient er grob 10...,[Als Vorstandsvorsitzender verdient er grob 10...,[[CLS] Der Autorin genügen fein gesetzte Motiv...,"[Ich habe zuvor nur so geschaltet, Ich lese ni..."
28309,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",['Baum : Ich mache das natürlich nicht mehr so...,98,[Als Vorstandsvorsitzender verdient er grob 10...,[Als Vorstandsvorsitzender verdient er grob 10...,"[[CLS] Doch Spirale II , das ebenfalls aus dem...","[Ich habe zuvor nur so geschaltet, Ich lese ni..."
28310,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",['Baum : Ich mache das natürlich nicht mehr so...,98,[Als Vorstandsvorsitzender verdient er grob 10...,[Als Vorstandsvorsitzender verdient er grob 10...,[[CLS] Als Vorstandsvorsitzender verdient er g...,"[Ich habe zuvor nur so geschaltet, Ich lese ni..."


In [43]:
llama_generated = final_merged[["construction", "evaluated_generated_llama"]].explode("evaluated_generated_llama").drop_duplicates()
llama_generated.to_csv(f"../../out/generated/llama_0_shot.tsv", sep="\t", decimal=",", index=False)
llama_generated

,construction,evaluated_generated_llama
0,10,Die meisten von ihnen haben zuvor noch nie ein...
0,10,Die meisten von ihnen haben zuvor noch nie ein...
27,100,Die beiden Pferde sind gleicher Größe.
27,100,Die beiden Mannschaften sind gleicher Stärke.
28,1004,Michael is the tallest
...,...,...
28311,99,Die Mobilität ist ungleichwertig
28311,99,Die Sprachen sind ungleichwertig
28311,99,Die Wetterbedingungen sind ungleichwertig
28311,99,Die Lebenserwartung ist ungleichwertig


In [44]:
llama_generated3 = final_merged[["construction", "evaluated_generated_llama3"]].explode("evaluated_generated_llama3").drop_duplicates()
llama_generated3.to_csv(f"../../out/generated/llama_3_shot.tsv", sep="\t", decimal=",", index=False)
llama_generated3

,construction,evaluated_generated_llama3
0,10,Die meisten von ihnen haben zuvor noch nie ein...
0,10,"Dennoch hat Hamas es nicht geschafft, ihre mil..."
0,10,"Reicher, geschweige denn glücklicher, wird ein..."
27,100,Die Schülerzahlen im Jahrgang A und B sind gle...
28,1004,Besser
...,...,...
28186,98,Das Leben ist hier sozusagen wunderbar
28186,98,Der Anruf war so unerfreulich wie erwartet
28311,99,Der Apple-iPhone ist im Vergleich zum Samsung-...
28311,99,Der neue Porsche-911-GT3-R-Sportwagen ist schn...


In [45]:
mbart_vanilla_generated = final_merged[["construction", "evaluated_generated_mbart_vanilla"]].explode("evaluated_generated_mbart_vanilla").drop_duplicates()
mbart_vanilla_generated.to_csv(f"../../out/generated/mbart_vanilla.tsv", sep="\t", decimal=",", index=False)
mbart_vanilla_generated

,construction,evaluated_generated_mbart_vanilla
0,10,"Die Chance dazu haben sie, und sie haben sie i..."
0,10,Sie kennen aber nicht ihre Ausgaben.
0,10,Ein verbesserter Zugang zu einzelnen Abgeordne...
9,10,Und dann ist da noch das generelle Problem mit...
9,10,"""Ohne Europa sind viele Fragen nicht mehr seriös"""
...,...,...
28286,98,"Der Baum, so wie er sein sollte."
28286,98,"Bei allen Abstimmungen, die so"
28286,98,"Sie machen so , was nützt."
28311,99,"im Gang sieht sie plötzlich Jamie, erkennt ihn..."


In [46]:
mbart_pseudo_generated = final_merged[["construction", "evaluated_generated_mbart_pseudo"]].explode("evaluated_generated_mbart_pseudo").drop_duplicates()
mbart_pseudo_generated.to_csv(f"../../out/generated/mbart_pseudo.tsv", sep="\t", decimal=",", index=False)
mbart_pseudo_generated

,construction,evaluated_generated_mbart_pseudo
0,10,"Die Chance dazu haben sie, und sie haben sie i..."
0,10,Sie kennen aber nicht ihren Kontostand.
0,10,..............
3,10,...........
3,10,»Ohne Europa sind viele Fragen nicht mehr seri...
...,...,...
28206,98,Sind die Noaks Marcumar?
28206,98,"Der Baum,.. sollte......"
28206,98,"Bei allen Abstimmungen, die Bevölkerung."
28206,98,Sie machen nützt.


In [47]:
bert_pseudo_generated = final_merged[["construction", "evaluated_generated_bert"]].explode("evaluated_generated_bert").drop_duplicates()
bert_pseudo_generated.to_csv(f"../../out/generated/bert_pseudo.tsv", sep="\t", decimal=",", index=False)
bert_pseudo_generated

,construction,evaluated_generated_bert
0,10,"[CLS] Geschweige10 denn , dass sie die Argumen..."
0,10,"[CLS] Geschweige10 denn , dass sie ihre Argume..."
0,10,"[CLS] Geschweige10 denn , dass sie die Autos e..."
0,10,"[CLS] Geschweige10 denn , dass sie die Argumen..."
0,10,"[CLS] Geschweige10 denn , dass sie die Argumen..."
...,...,...
28190,98,"[CLS] Sie machen so98 nicht mit , wie es ihnen..."
28190,98,"[CLS] Sie machen so98 lange nichts , wie es ih..."
28190,98,"[CLS] Sie machen so98 lange mit , weil es ihne..."
28190,98,"[CLS] Sie machen so98 lange mit , wie es ihnen..."
